In [ ]:
#!pip install -q kaggle

In [ ]:
'''from google.colab import files
files.upload()'''

In [ ]:
'''!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json
!kaggle competitions download -c tuberculosis-drug-resistance-prediction-cap'''

  0% 0.00/74.1k [00:00<?, ?B/s]
100% 74.1k/74.1k [00:00<00:00, 28.7MB/s]
  0% 0.00/443k [00:00<?, ?B/s]
100% 443k/443k [00:00<00:00, 60.7MB/s]
  0% 0.00/91.0k [00:00<?, ?B/s]
100% 91.0k/91.0k [00:00<00:00, 81.3MB/s]
  0% 0.00/2.84k [00:00<?, ?B/s]
100% 2.84k/2.84k [00:00<00:00, 2.92MB/s]


In [ ]:
'''!unzip ./X_trainData_1.csv.zip
!rm ./X_trainData_1.csv.zip'''

Archive:  ./X_trainData_1.csv.zip
  inflating: X_trainData_1.csv       


In [ ]:
import pandas as pd
df = pd.read_csv('X_trainData_1.csv')
#df

In [ ]:
df1 = pd.read_csv('Y_trainData_1.csv')
#df1

In [ ]:
for i in df1.columns:
  print(df1[i].value_counts())

In [ ]:
df = pd.read_csv('X_trainData_1.csv')
df1 = pd.read_csv('Y_trainData_1.csv')

df.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
df.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
df.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)
count=0
for ind in df1.index:
  x = df1['CAP'][ind]
  if x == -1:
    count+=1
    df.drop(index=ind,inplace=True)
    df1.drop(index=ind,inplace=True)
print(count)
df['CAP'] = df1['CAP']



2054


In [ ]:
## Correlation
#Trying to find the correlation between the feature columns and the target variable for Dimensionality Reduction.Didn't workout
'''
import seaborn as sns
import matplotlib.pyplot as plt
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")
correlation = corrmat['CAP'].sort_values()
correlation = correlation.iloc[:10]
df = df.drop(columns=correlation.index,axis=1)'''

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
import numpy as np
import pickle

In [ ]:
ss = StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=42)

traindata1 = df.drop(columns=['CAP'],axis=1)
y1 = df['CAP']

for train_index, test_index in ss.split(traindata1,y1):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = traindata1.iloc[train_index], traindata1.iloc[test_index]
  y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]


X_train.shape,X_test.shape

((1205, 219), (134, 219))

In [ ]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.01,0.03,0.05, 0.10, 0.15, 0.20, 0.25, 0.30,0.35,0.4] ,
 "max_depth"        : [ 6,8, 10, 12,14, 15,16],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350,370,400]   
}
params1 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,5,1.6,1.7,1.9,2,2.2,2.4],
   'kernel' : ['rbf','linear'],
   'degree':[1,2,3,5,6],
    'class_weight':['balanced',''],
  'decision_function_shape':['ovo', 'ovr']
}
params2 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,3,1,5,1.6,1.7,1.9,2,2.2],
    'class_weight':['balanced',''],
    'solver':['newton-cg', 'liblinear', 'sag', 'saga'],
    'max_iter':[20,40,60,80,100,120,140,180,200,230,250,270,300]
}
params3 = {
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_samples_split" : [ 1,2, 3,4, 5,6, 7 ],
 "max_features" : [ 'auto','sqrt','log2' ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350]   
}

In [ ]:
classifier= SVC()
random_search=RandomizedSearchCV(classifier,param_distributions=params2,n_iter=25,scoring='roc_auc'
                                 ,n_jobs=-1,cv=2,verbose=3)

grid_search=GridSearchCV(classifier,param_grid=params1,scoring='roc_auc',
                                 n_jobs=-1,cv=2,verbose=3)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
print(grid_search.best_estimator_)
grid_search.best_score_

In [ ]:
lr = LogisticRegression(C=0.3, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8582089552238806
[[52  7]
 [12 63]]
              precision    recall  f1-score   support

           0       0.81      0.88      0.85        59
           1       0.90      0.84      0.87        75

    accuracy                           0.86       134
   macro avg       0.86      0.86      0.86       134
weighted avg       0.86      0.86      0.86       134



In [ ]:
svm = SVC(gamma='scale',C=1.7,kernel='rbf',class_weight='balanced',probability=True)


svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9253731343283582
[[57  2]
 [ 8 67]]
              precision    recall  f1-score   support

           0       0.88      0.97      0.92        59
           1       0.97      0.89      0.93        75

    accuracy                           0.93       134
   macro avg       0.92      0.93      0.92       134
weighted avg       0.93      0.93      0.93       134



In [ ]:
rf = RandomForestClassifier(n_estimators=50,max_depth=14)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

y_pred = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9104477611940298
[[54  5]
 [ 7 68]]
              precision    recall  f1-score   support

           0       0.89      0.92      0.90        59
           1       0.93      0.91      0.92        75

    accuracy                           0.91       134
   macro avg       0.91      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [ ]:
xf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              validate_parameters=1)

xf.fit(X_train, y_train)

print(xf.score(X_train, y_train))
print(xf.score(X_test, y_test))
y_pred = xf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9800829875518672
0.8731343283582089
[[48 11]
 [ 6 69]]
              precision    recall  f1-score   support

           0       0.89      0.81      0.85        59
           1       0.86      0.92      0.89        75

    accuracy                           0.87       134
   macro avg       0.88      0.87      0.87       134
weighted avg       0.87      0.87      0.87       134



**Trying Neural Network**

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.utils import class_weight
from keras import optimizers
tf.__version__


'2.3.0'

In [ ]:
X_train.shape

(1205, 219)

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.InputLayer(input_shape=219),
tf.keras.layers.Dense(32,activation='relu'),
#tf.keras.layers.Dropout(0.1),
tf.keras.layers.Dense(16,activation='relu'),
tf.keras.layers.Dense(8,activation='relu'),
tf.keras.layers.Dense(4,activation='relu'),
tf.keras.layers.Dense(1,activation='sigmoid'),                                  
])


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),
              )
model.summary()
weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(weights)
temp = {0:weights[0],1:weights[1]}
type(temp)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 32)                7040      
_________________________________________________________________
dense_46 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_47 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_48 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 5         
Total params: 7,745
Trainable params: 7,745
Non-trainable params: 0
_________________________________________________________________
[1.14109848 0.88995569]


dict

In [ ]:
#!unzip model_CAP.zip
#!rm model_CAP.zip
model = tf.keras.models.load_model('./model_CAP/')
model.summary()

Archive:  model_CAP.zip
replace model/variables/variables.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: model/variables/variables.data-00000-of-00001  
  inflating: model/variables/variables.index  
  inflating: model/saved_model.pb    
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 32)                7040      
_________________________________________________________________
dense_23 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_24 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_25 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_26 (Dense)             (None, 1)               

In [ ]:
history = model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test),class_weight=temp)

38/38 [==============================] - 0s 5ms/step - loss: 0.0627 - val_loss: 0.3365


In [ ]:
model.save('model_CAP')
!zip -r model_CAP.zip ./model_CAP

INFO:tensorflow:Assets written to: model/assets
  adding: model/ (stored 0%)
  adding: model/assets/ (stored 0%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 16%)
  adding: model/variables/variables.index (deflated 65%)
  adding: model/saved_model.pb (deflated 89%)


In [ ]:
preds = model.predict(X_test)
preds1 = np.where(preds>0.5,1,0)
preds2 = np.where(model.predict(X_train)>0.5,1,0)
print(np.unique(preds1,return_counts=True))
print(accuracy_score(y_train,preds2))
print(accuracy_score(y_test,preds1))
print(confusion_matrix(y_test,preds1))
print(roc_auc_score(y_test,preds))

(array([0, 1]), array([62, 72]))
0.9452282157676348
0.8582089552238806
[[51  8]
 [11 64]]
0.943954802259887


In [ ]:
print('hello')
#!rm -r ./model/
#!rm -r ./model1/
#!rm ./model.zip

hello


In [ ]:
X_testdata = pd.read_csv('X_testData_1.csv')
Y_testdata_nolabel = pd.read_csv('Y_testData_1_nolabels_CAP.csv')
output_id = set(Y_testdata_nolabel['ID'])
len(output_id)

501

In [ ]:
generate_id = X_testdata['ID']
X_testdata = X_testdata[X_train.columns]
#X_testdata.drop('ID',axis=1,inplace=True)
#generate_id
#X_testdata

In [ ]:
#Run this to produce output for ML models
output_pred = lr.predict_proba(X_testdata)
out = pd.DataFrame({'CAP':output_pred[:,1]})
out['ID'] = generate_id
out = out[['ID','CAP']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
out['CAP'].value_counts()

0.973697    22
0.970582     9
0.977264     7
0.962019     5
0.973683     4
            ..
0.520113     1
0.983224     1
0.964689     1
0.937619     1
0.867069     1
Name: CAP, Length: 407, dtype: int64

In [ ]:
#Run this to produce output for Neural Network
output_pred = model.predict(X_testdata)
out = pd.DataFrame({'CAP':output_pred[:,0]})
out['ID'] = generate_id
out = out[['ID','CAP']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
out['CAP'].value_counts()

In [ ]:
out.to_csv('Group_7_output.csv',index=None)